In [1]:
#https://github.com/ageek/langchain-tutorials
#https://github.com/gkamradt/langchain-tutorials/blob/main/LangChain%20Cookbook%20Part%201%20-%20Fundamentals.ipynb

In [2]:
from dotenv import load_dotenv

In [3]:
load_dotenv("/home/tom/two/envapi/my-env")

True

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

In [5]:
chat = ChatOpenAI(
    temperature=0.7
)

In [7]:
chat(
    [
        SystemMessage(content="You are a assistant that helps where a user wants to go. Limit your response to 80 words maximum.Be clear in your response."),
        HumanMessage(content="I like beaches, where should I go"),
        AIMessage(content="You should go to Goa, India"),
        HumanMessage(content="What else should I do there?")
        
    ]
)

AIMessage(content="In Goa, you can also visit the historic churches, explore the vibrant local markets, try water sports like snorkeling and jet skiing, and indulge in delicious seafood. Don't miss the opportunity to witness the mesmerizing sunset at the beach and experience the lively nightlife.")

In [9]:
chat(
    [
        HumanMessage(content="I like paragliding, where to go?")
    ]
)

AIMessage(content='There are numerous excellent locations worldwide for paragliding. Here are some popular destinations:\n\n1. Interlaken, Switzerland: Surrounded by the Swiss Alps, Interlaken offers breathtaking views and a variety of launch sites for all skill levels.\n\n2. Pokhara, Nepal: Known as the paragliding capital of Nepal, Pokhara offers stunning views of the Himalayas and the serene Phewa Lake.\n\n3. Oludeniz, Turkey: With its turquoise blue waters and stunning landscapes, Oludeniz is a popular spot for paragliding. The Babadag Mountain provides an excellent launching point.\n\n4. Queenstown, New Zealand: This adventure capital offers incredible paragliding opportunities with stunning views of Lake Wakatipu and the surrounding mountains.\n\n5. Cape Town, South Africa: Paragliding over Cape Town allows you to soar above the beautiful beaches, Table Mountain, and the Atlantic Ocean.\n\n6. Rio de Janeiro, Brazil: Flying over the iconic city of Rio de Janeiro offers breathtakin

In [10]:
chat(
    [
        HumanMessage(content="What comes after Sunday?")
    ]
)

AIMessage(content='After Sunday comes Monday.')

In [11]:
#language models

In [12]:
from langchain.llms import OpenAI

In [13]:
llm = OpenAI(
    model="text-ada-001"
)
llm("What comes after Sunday?")

'\n\nMonday is the next day in history.'

In [14]:

#function calling

In [15]:
chat  = ChatOpenAI(
    model = "gpt-3.5-turbo",
    temperature=.9
)

output = chat(messages=
              [
                  SystemMessage(content="You are a helpful AI bot"),
                  HumanMessage(content="What is the weather like in Boston right now?")
              ],
              functions=[{
                  "name":"get_current_weahter",
                  "description":"Get the weather for a given location",
                  "parameters":{
                      "type":"object",
                      "properties":{
                          "location":{
                              "type":"string",
                              "description":"The city and state, e.g San Francisco, CA"
                          },
                          "unit":{
                              "type":"string",
                              "enum":["celcius", "fahrenheit"]
                          }
                      },
                      "required":["location"]
                  }
              }
              ]
             
)

output

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_weahter', 'arguments': '{\n  "location": "Boston, MA"\n}'}})

In [16]:
#Text Embedding models

from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [17]:
text = "Hey, there ! I'm flying to Dubai next week."

In [18]:
text_embeddings = embeddings.embed_query(text)
print(f"Here is a sample: {text_embeddings[:20]}")
print(f"Embedding lenth {len(text_embeddings)}")
      

Here is a sample: [-0.021728801837556426, -0.021400914805473653, -0.01291369254098899, -0.02731548776604596, -0.010486069727913473, 0.02759293063934677, -0.03185545833113234, -0.015322399260182584, 0.0030219177094795617, -0.023229514629997273, 0.021880134313902323, -0.007787310027046206, 0.011091399633297059, -0.027845149570611342, 0.021602691440601514, -0.02447800662852829, 0.021968410660448134, -0.025285111306394478, 0.017478884254143724, -0.009546548667583308]
Embedding lenth 1536


In [19]:
#Prompts
from langchain.llms import OpenAI

In [21]:
llm = OpenAI(
    temperature=0.9, 
    model="text-davinci-003"
)

prompt="""
    Today is Friday and Tomorrow is Sunday. 
    What is wrong with the statment?
"""

print(llm(prompt))


The statement is incorrect because tomorrow is Saturday, not Sunday.


In [22]:
#prompt template

In [23]:
from langchain.llms import OpenAI

In [24]:
from langchain import PromptTemplate

In [28]:
llm = OpenAI(
    temperature=.9, 
    model="text-davinci-003",
)

template = """
    I want to travel to {location}. What should I do there? Respond in one sentece. 
"""

prompt = PromptTemplate(
    input_variables=["location"],
    template= template
)

final_prompt = prompt.format(location="Dhaka")
print(f"Final prompt: {final_prompt}")
print("=============================")
print(f"LLM output: {llm(final_prompt)}")

Final prompt: 
    I want to travel to Dhaka. What should I do there? Respond in one sentece. 

LLM output: 
Explore the history and culture of Dhaka and visit some of its famous landmarks.


In [29]:


#Example selectors

In [37]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.llms import OpenAI


In [62]:
llm = OpenAI(
    temperature=.9, 
    model="text-davinci-003"
    #model="gpt-3.5-turbo"
)

example_prompt=PromptTemplate(
    input_variables=["input", "output"],
    template="Example Input : {input}\nExample Output: {output}"
)

examples = [
    {"input":"pirate", "output":"ship"},
    {"input":"pilot", "output":"plane"},
    {"input":"driver", "output":"car"},
    {"input":"tree", "output":"ground"},
    {"input":"bird", "output":"nest"}
]


In [63]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples, 
    OpenAIEmbeddings(),
    Chroma, 
    k=2
)

In [64]:
example_selector

SemanticSimilarityExampleSelector(vectorstore=<langchain.vectorstores.chroma.Chroma object at 0x7fe4caa8aa70>, k=2, example_keys=None, input_keys=None)

In [65]:
similar_prompt = FewShotPromptTemplate(
    example_selector= example_selector,
    example_prompt= example_prompt,
    prefix="Given the location an item is usually found in-",
    suffix="Input: {noun}\nOutput: ",
    input_variables=["noun"],
)

In [69]:
my_noun="student"
print(similar_prompt.format(noun=my_noun))

Given the location an item is usually found in-

Example Input : driver
Example Output: car

Example Input : driver
Example Output: car

Input: student
Output: 


In [70]:
noun_list=["rabbit"]
import time
for noun in noun_list:
    #print(similar_prompt.format(noun=noun))
    print(llm(similar_prompt.format(noun=noun)))
    #time.sleep(20)

 burrow


In [67]:
#Output Parsers Method 1: Prompt Instructions & String Parsing

In [68]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI

In [71]:
llm = OpenAI(
    model="text-davinci-003",
    temperature=0.9
)

In [76]:
response_schema = [
    ResponseSchema(name="bad_string", description="This is poorly formatted user input string."),
    ResponseSchema(name="good_string", description="This is your response, a reformatted response.")
]
output_parser = StructuredOutputParser.from_response_schemas(response_schema)
output_parser

StructuredOutputParser(response_schemas=[ResponseSchema(name='bad_string', description='This is poorly formatted user input string.', type='string'), ResponseSchema(name='good_string', description='This is your response, a reformatted response.', type='string')])

In [74]:
format_instructions = output_parser.get_format_instructions()
format_instructions
#output_parser.get_format_instructions
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"bad_string": string  // This is poorly formatted user input string.
	"good_string": string  // This is your response, a reformatted response.
}
```


In [77]:
template="""
    You will be a given a poorly formatted string from user. Reformat it
    and make sure all the words are spelled correctly. 
    
    {format_instructions}
    
    {user_input}
    
    Your_response:
"""

prompt = PromptTemplate(
    input_variables=["user_input"],
    partial_variables={"format_instructions": format_instructions},
    template=template
)

promptValue = prompt.format(user_input="wlecome to deli, india?")
print(promptValue)


    You will be a given a poorly formatted string from user. Reformat it
    and make sure all the words are spelled correctly. 
    
    The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"bad_string": string  // This is poorly formatted user input string.
	"good_string": string  // This is your response, a reformatted response.
}
```
    
    wlecome to deli, india?
    
    Your_response:



In [79]:
llm_output = llm(promptValue)
print(llm_output)

```json
{
	"bad_string": "wlecome to deli, india?",
	"good_string": "Welcome to Delhi, India?"
}
```


In [80]:
output_parser.parse(llm_output)

{'bad_string': 'wlecome to deli, india?',
 'good_string': 'Welcome to Delhi, India?'}

In [81]:
#Output Parsers Method 2: OpenAI Fuctions

In [82]:
from langchain.pydantic_v1 import BaseModel, Field
from typing import Optional

In [83]:
class Person(BaseModel):
    name: str = Field(..., description="the person's name")
    age: int = Field(..., description="the persons age")
    fav_food:Optional[str] = Field(None, description="the person's favorite food")

In [84]:
from langchain.chains.openai_functions import create_structured_output_chain
llm = OpenAI(
    model='gpt-3.5-turbo',
    temperature=0.9
)

chain = create_structured_output_chain(Person, llm, prompt)

chain.run(
    "Rosell is 29, Joe just turned 18 and loves pizza. Caroline is 10 years older than Rosell and likes prawns."
)

InvalidRequestError: Unrecognized request arguments supplied: function_call, functions

In [93]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from langchain.chains import SimpleSequentialChain

In [95]:
#output from bing.com/chat gpt-4 based search result

model = ChatOpenAI()
text_prompt = PromptTemplate.from_template(
    """
    Write a short text about {topic}.
    Text:
    """
)
text_chain = text_prompt | ChatOpenAI()
summary_prompt = PromptTemplate.from_template(
    """
    Write a summary of the text in one sentence.
    Text: {text}
    Summary:
    """
)
summary_chain = summary_prompt | ChatOpenAI()

sentiment_prompt = PromptTemplate.from_template(
    """
    Write the sentiment of the text as positive, negative, or neutral.
    Text: {text}
    Sentiment:
    """
)
sentiment_chain = sentiment_prompt|ChatOpenAI()

full_chain = SimpleSequentialChain(
    [{"text": text_prompt | model | StrOutputParser()} |
     {"summary": summary_prompt | model | StrOutputParser()} |
     {"sentiment": sentiment_prompt | model | StrOutputParser()}]
)
# chain = (
#     {"text": text_prompt | model | StrOutputParser()} |
#     {"summary": summary_prompt | model | StrOutputParser()} |
#     {"sentiment": sentiment_prompt | model | StrOutputParser()}
# )

full_chain.invoke({"topic": "sports"})

TypeError: Serializable.__init__() takes 1 positional argument but 2 were given

In [96]:
#Document Loaders

In [97]:
from langchain.document_loaders import HNLoader

In [98]:
loader= HNLoader("https://news.ycombinator.com/item?id=34422627")

In [99]:
data = loader.load()

In [100]:
print(f"Found {len(data)} comments")

Found 76 comments


In [107]:
for x in data[:3]:
    print(x.page_content)
    
' '.join(x.page_content[:150] for x in data)    

Ozzie_osman 9 months ago  
             | next [–] 

LangChain is awesome. For people not sure what it's doing, large language models (LLMs) are very powerful but they're very general. As a common example for this limitation, imagine you want your LLM to answer questions over a large corpus.You can't pass the entire corpus into the prompt. So you might:
- preprocess the corpus by iterating over documents, splitting them into chunks, and summarizing them
- embed those chunks/summaries in some vector space
- when you get a question, search your vector space for similar chunks
- pass those chunks to the LLM in the prompt, along with your questionThis ends up being a very common pattern, where you need to do some preprocessing of some information, some real-time collecting of pieces, and then an interaction with the LLM (in some cases, you might go back and forth with the LLM). For instance, code and semantic search follows a similar pattern (preprocess -> embed -> nearest-neighbors at que

'Ozzie_osman 9 months ago  \n             | next [–] \n\nLangChain is awesome. For people not sure what it\'s doing, large language models (LLMs) are very  Ozzie_osman 9 months ago  \n             | parent | next [–] \n\nAlso, another library to check out is GPT Index (https://github.com/jerryjliu/gpt_index) fireant 9 months ago  \n             | root | parent | next [–] \n\nThank you for the link. I have a project with plenty of structured data from which I  bluecoconut 9 months ago  \n             | prev | next [–] \n\nThis is great! I love seeing how rapidly in the past 6 months these ideas are evolving. I\' gandalfgeek 9 months ago  \n             | parent | next [–] \n\nLambdaPrompt looks really cool. Love the Pythonic expression as good ol\' functions, whic bluecoconut 9 months ago  \n             | root | parent | next [–] \n\nIn terms of applications with it, I have made things like sketch: https://github. clxy 9 months ago  \n             | root | parent | next [–] \n\nThanks 

In [108]:
#gutenberg loaders
from langchain.document_loaders import GutenbergLoader

loader = GutenbergLoader("https://www.gutenberg.org/cache/epub/2148/pg2148.txt")
data = loader.load()
print(f"data of len {len(data)} found")

data of len 1 found


In [116]:
data[0].page_content[30000:30567]

'd what I meant in suggesting that, had the purloined\r\n\n\n      letter been hidden any where within the limits of the Prefect’s\r\n\n\n      examination—in other words, had the principle of its concealment\r\n\n\n      been comprehended within the principles of the Prefect—its\r\n\n\n      discovery would have been a matter altogether beyond question.\r\n\n\n      This functionary, however, has been thoroughly mystified; and the\r\n\n\n      remote source of his defeat lies in the supposition that the\r\n\n\n      Minister is a fool, because he has acquired renown as a poet. All\r\n\n\n    '

In [115]:
len(data[0].page_content)

653528

In [125]:
type(data[0].page_content)

str

In [117]:
#Text Splitters

In [118]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [120]:
import urllib.request
with urllib.request.urlopen("https://www.gutenberg.org/cache/epub/2148/pg2148.txt") as f: 
    pg_work = f.read()

print(f"You have {len(pg_work)} size document")

You have 638530 no of pages


In [122]:
pg_work[34839:34999]

b', the Prefect would have been under no necessity of\r\n      giving me this check. I know him, however, as both mathematician\r\n      and poet, and my measures wer'

In [127]:
type(pg_work.decode('utf-8'))

str

In [128]:
data = pg_work.decode('utf-8')

In [129]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200, 
    chunk_overlap=50
)

texts = text_splitter.create_documents([data])

In [131]:
len(texts)

4288

In [133]:
print(texts[4287].page_content)

including how to make donations to the Project Gutenberg Literary
Archive Foundation, how to help produce our new eBooks, and how to
subscribe to our email newsletter to hear about new eBooks.


In [134]:
print(texts[3333].page_content)

foot in the gorgeous bignonia blossoms that one required no
      little scrutiny to determine what manner of sweet thing it could


In [135]:
#Retrievers

In [136]:
import urllib.request
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

In [144]:
with urllib.request.urlopen("https://www.gutenberg.org/cache/epub/2148/pg2148.txt") as f: 
    pg_bytes = f.read()

pg_data= pg_bytes.decode('utf-8')

In [139]:
print(f"read {len(pg_data)} bytes of data")

read 632145 bytes of data


In [146]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, 
    chunk_overlap=100
)
texts = text_splitter.create_documents([pg_data])

embeddings = OpenAIEmbeddings()

db = Chroma.from_documents(texts, embeddings)

In [147]:
retriever = db.as_retriever()

In [148]:
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain.vectorstores.chroma.Chroma object at 0x7fe4b643a710>)

In [149]:
docs = retriever.get_relevant_documents("what turned my attacks")


In [151]:
print(' '.join([x.page_content for x in docs[:2]]))

be no more. From these latter attacks I awoke, however, with a
      gradation slow in proportion to the suddenness of the seizure.
      Just as the day dawns to the friendless and houseless beggar who
      roams the streets throughout the long desolate winter night—just
      so tardily—just so wearily—just so cheerily came back the light
      of the Soul to me.

      Apart from the tendency to trance, however, my general health came in my way. But my disease grew upon me—for what disease is
      like Alcohol!—and at length even Pluto, who was now becoming old,
      and consequently somewhat peevish—even Pluto began to experience
      the effects of my ill temper.

      One night, returning home, much intoxicated, from one of my
      haunts about town, I fancied that the cat avoided my presence. I
      seized him; when, in his fright at my violence, he inflicted a


In [152]:
#Vectorestores

In [153]:
#improts etc from last section

In [154]:
embedding_list = embeddings.embed_documents([x.page_content for x in docs])


In [155]:
len(embedding_list)

4

In [156]:
print(f"sample embedding: {embedding_list[0][:10]}")

sample embedding: [0.0001422118558829291, -0.001656336997250217, 0.021528015622371156, -0.03573566792905587, -0.008675468440036412, 0.026724914969209868, 0.009709260746301428, -0.040848744516127626, -0.026417571058881727, -0.0010826873251196236]


In [157]:
#Memory

#Helping LLMs remember information.

In [158]:
#example - chat msg history
from langchain.memory import ChatMessageHistory
from langchain.chat_models import ChatOpenAI

In [159]:
chat = ChatOpenAI(
    temperature=0.9
)

history  = ChatMessageHistory()
history.add_ai_message("Hi")
history.add_user_message("whats the capital of Srilanka?")

In [160]:
history.messages

[AIMessage(content='Hi'),
 HumanMessage(content='whats the capital of Srilanka?')]

In [161]:
ai_response = chat(history.messages)
ai_response

AIMessage(content='The capital of Sri Lanka is Colombo.')

In [163]:
history.add_ai_message(ai_response.content)
history.messages

[AIMessage(content='Hi'),
 HumanMessage(content='whats the capital of Srilanka?'),
 AIMessage(content='The capital of Sri Lanka is Colombo.')]

In [164]:
#Chains
#Combining different LLM calls and action automatically

#Ex: Summary #1, Summary #2, Summary #3 > Final Summary

In [181]:
#1. Simple Sequential Chains
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SimpleSequentialChain

In [166]:
llm = OpenAI(temperature=0.9)

In [171]:
template = """
    You are an assistant that tells the country a given {user_location} belongs to.
    
    input: {user_location}
    Your Response:
"""

prompt_template = PromptTemplate(
    input_variables=["user_location"],
    template=template
)

location_chain = LLMChain(
    llm=llm, 
    prompt=prompt_template
)

In [172]:
template = """
    Given a country, list the top 5 tourist attractions of that country.
    input: {country}
    Your Response:
"""
prompt_template = PromptTemplate(
    input_variables=["country"],
    template=template
)

country_chain = LLMChain(
    llm=llm, 
    prompt=prompt_template
)

In [173]:
overall_chain = SimpleSequentialChain(
    chains=[location_chain, country_chain],
    verbose=True
)


In [175]:
review = overall_chain.run("Nurenberg")



> Entering new SimpleSequentialChain chain...
Nuremberg is a city in Germany.
1. Nuremberg Castle: This historic castle dates back to the 11th century and has acted as a fortification, royal residence, and prison.
2. Germanisches Nationalmuseum: This museum is the largest museum of German culture and art in the world and features a wide variety of items from prehistoric to modern times.
3. Nazi Rally Grounds: This site was the birthplace of the infamous Nazi rallies and features monuments and memorials to commemorate this dark history.
4. Hauptmarkt: This city center is home to a weekly market and popular for its shops, restaurants, and Christmas markets.
5. St. Sebaldus Church: This hundreds of years old church is one of the oldest in the region and features unique architecture.

> Finished chain.


In [176]:
template = """Given a list of 5 tourist places, randomly pick one and suggest the famous dish of that locality
    input: {tourist_place}
    Your Response:
"""
prompt_template = PromptTemplate(
    input_variables=["tourist_place"],
    template=template
)
tourist_place_chain = LLMChain(
    llm = llm, 
    prompt=prompt_template
)


In [178]:
overall_chain = SimpleSequentialChain(
    chains=[location_chain, country_chain, tourist_place_chain], 
    verbose=True
)

In [180]:
overall_chain.run("Agra")



> Entering new SimpleSequentialChain chain...
Agra is located in India.


Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-pVOnO8dpEcNyuE7hDq8Ei5vA on requests per min. Limit: 3 / min. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..



1. Taj Mahal - Agra, India 
2. Agra Fort - Agra, India 
3. Fatehpur Sikri - Agra, India 
4. Itimad-Ud-Daulah - Agra, India 
5. Mughal Heritage Walk - Agra, India


Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-pVOnO8dpEcNyuE7hDq8Ei5vA on requests per min. Limit: 3 / min. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-pVOnO8dpEcNyuE7hDq8Ei5vA on requests per min. Limit: 3 / min. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.llms.openai.completion_with_retry.<locals

KeyboardInterrupt: 

In [182]:
#2. Summarization Chain

In [183]:
url="https://raw.githubusercontent.com/run-llama/llama_index/main/examples/paul_graham_essay/data/paul_graham_essay.txt"

In [184]:
from urllib.request import urlopen

with urlopen(url) as f:
    page_bytes = f.read()
    
page_str = page_bytes.decode("utf-8")


In [185]:
type(page_str)

str

In [186]:
len(page_str)

75014

In [189]:
page_str[:200]

"\n\nWhat I Worked On\n\nFebruary 2021\n\nBefore college the two main things I worked on, outside of school, were writing and programming. I didn't write essays. I wrote what beginning writers were supposed "

In [190]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain

In [193]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=200
)
texts = text_splitter.create_documents([page_str])

In [195]:
len(texts)

105

In [196]:
chain = load_summarize_chain(
    llm=llm, 
    chain_type='map_reduce', 
    verbose=True
)
#to avoid rate limiting use only first 10
chain.run(texts[:10])



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"What I Worked On

February 2021

Before college the two main things I worked on, outside of school, were writing and programming. I didn't write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.

The first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district's 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain's lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised fl


> Finished chain.

> Finished chain.


' This article tells the story of a novice writer who, before college, taught themselves how to program with Fortran on an IBM 1401. After convincing their father to buy a TRS-80, the narrator found a more interesting topic to study in college: Artificial Intelligence (AI). When their diploma came with "AI" in scare-quotes, it became amusingly accurate in hindsight. During graduate school at Harvard, the narrator realized that the existing AI programs of the time were a hoax.'

In [197]:
#Agents
#Some applications will require not just a predetermined chain of calls to LLMs/other tools, 
#but potentially an unknown chain that depends on the user's input. In these types of chains,
#there is a “agent” which has access to a suite of tools. Depending on the user input, the agent
#can then decide which, if any, of these tools to call.

#Basically you use the LLM not just for text output, but also for decision making. 
#The coolness and power of this functionality can't be overstated enough.

In [207]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.llms import OpenAI
from langchain.agents import AgentType
import json
import os

In [202]:
load_dotenv('/home/tom/two/envapi/my-env')

True

In [203]:
llm = OpenAI(
    temperature=0.1
)


In [205]:
toolkit = load_tools(
    ['serpapi'], 
    llm=llm
)

In [211]:
agent = initialize_agent(
    toolkit, 
    llm=llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, 
    return_intermediate_steps=True,
    verbose=True
)

In [209]:
resonse = agent(
    {"input":"what was the first album of the band that alish chinoy was part of ?"}
)



> Entering new AgentExecutor chain...
Thought: I need to find out what band Alish Chinoy was part of
Action:
```
{
  "action": "Search",
  "action_input": "Alish Chinoy band"
}
```

Observation: ['Alisha Chinai (born 18 March 1965) is an Indian pop singer known for her Indi-pop albums as well as playback singing in Hindi cinema. ... She began her singing ...', "Made in India, is the undisputed Queen of Indian Pop! She blazed a trail of chart busters n ignited a pop revolution in the mid 90's... Her Pop Anthem 'Made in ...", '70K Followers, 947 Following, 1819 Posts - See Instagram photos and videos from Alisha Chinai (@alishachinaiofficial)', 'Indian pop singer, she has been active since 1985. She has released her own albums and also sung Hindi songs for many Bollywood films over the years.', 'Alisha Chinai discography and songs: Music profile for Alisha Chinai, born 18 March 1965. Genres: Indian Pop, Synthpop, Dance-Pop.', "Singer Alisha Chinai has found love once again, and this ti

In [214]:
resonse = agent(
    {"input":"What was the first movie of the actor playing the character of Akbar in the movie Amar Akbar Anthony"}
)



> Entering new AgentExecutor chain...

Thought: I need to find out the actor's name and then search for the first movie they were in
Action:
```
{
  "action": "Search",
  "action_input": "actor playing Akbar in Amar Akbar Anthony"
}
```

Observation: Amar (Vinod Khanna) has followed in the footsteps of his adoptive father to become a dashing and exemplary (which is to say, tough) police officer, and Akbar (Rishi Kapoor) is a passionate qawwali singer in love with a young doctor named Salma (Neetu Singh).
Thought: I need to find out the actor's name
Action:
```
{
  "action": "Search",
  "action_input": "actor playing Akbar in Amar Akbar Anthony"
}
```


Observation: Amar (Vinod Khanna) has followed in the footsteps of his adoptive father to become a dashing and exemplary (which is to say, tough) police officer, and Akbar (Rishi Kapoor) is a passionate qawwali singer in love with a young doctor named Salma (Neetu Singh).
Thought: I need to find out the actor's first movie
Action:
```
{

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-pVOnO8dpEcNyuE7hDq8Ei5vA on requests per min. Limit: 3 / min. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..


 I now know the actor's first movie
Final Answer: Rishi Kapoor's first movie was Mera Naam Joker (1970).

> Finished chain.


In [218]:
#print(json.dumps(resonse['intermediate_steps'], indent=2))

/bin/bash: line 1: [amar: command not found
